## Импорт библиотек

In [1]:
import pandas as pd
import os
import warnings
import numpy as np
import re
import json

from functools import reduce
from datetime import datetime
from datetime import date

warnings.filterwarnings("ignore")

## Определение функций

In [2]:
def bodyType_unification(x):
    res=[]
    x = x.lower() if type(x) == str else x  # <================
    try:
        for body_type in test.bodyType.unique():
            if body_type in x:
                res.append(body_type)
    except Exception: return x
    return max(res)

def engineDisplacement_unification(x):
    x = float(re.sub("[^\d.]", r'', x)) if re.sub("[^\d.]", r'', x) else 0
    if x >= 7: x = 0
    return x

def train_options(x):
    point = "'available_options': "
    start = x.find(point)+len(point)+2
    finish = x.find("]",start) - 1
    return x[start:finish].split("', '")

def test_ownership_unification(x):
    try:
        digits = re.findall('\d+',x) 
        if len(digits) == 2: res = int(digits[0])*12 + int(digits[1])
        elif len(digits) == 1 and 'месяц' in x: res = int(digits[0])
        elif len(digits) == 1 and 'месяц' not in x: res = int(digits[0])*12
    except Exception: res = 0   
    return res

def train_ownership_unification(x):
    tmp = json.loads(x.replace("'",'"'))  if x==x else {'year': 2020, 'month': 9}
    res = (2020 - tmp['year'])*12 + tmp['month'] - 9
    if res<0: res = 0
    return res

def test_unification(df_to_proc):
    test = df_to_proc.copy()[test_0909_uni_test]
    owner_dict = {'3 или более': 3., '2\xa0владельца': 2.,'1\xa0владелец': 1.}
    # engineDisplacement
    test.engineDisplacement = test.engineDisplacement.apply(
        lambda x: float(x[:-4]) if x[:-4] else np.NaN) 
    # enginePower
    test.enginePower = test.enginePower.apply(lambda x: int(x[:-4]))
    # equipment_dict
    test.equipment_dict = test.equipment_dict.apply(
        lambda x: list(json.loads(x).keys()) if x==x else [])
    # Владельцы
    test.Владельцы = test.Владельцы.map(owner_dict)
    # Владение
    test.Владение = test.Владение.apply(test_ownership_unification)
    return test

def train_0909_unification(df_to_proc):
    df = df_to_proc.copy()[test_0909_uni_0909]
    color_codes = {'040001': 'чёрный','FAFBFB': 'белый', '0000CC': 'синий', 
                   '200204': 'коричневый', 'EE1D19': 'красный', 'CACECB': 'серый',
                   'C49648': 'бежевый', '97948F': 'серебристый', 'FFD600': 'жёлтый',
                   'FF8649': 'оранжевый', '22A0F8': 'голубой','FFC0CB': 'розовый', 
                   'DEA522': 'золотистый', '007F00': 'зелёный', '660099': 'пурпурный',
                   '4A2197': 'фиолетовый'}
    transmission_dict = {'MECHANICAL':'механическая', 'AUTOMATIC':'автоматическая', 
                         'ROBOT':'роботизированная','VARIATOR':'вариатор'}
    vendor_dict = {'AUDI':'EUROPEAN','BMW':'EUROPEAN','HONDA':'JAPANESE','INFINITI':'JAPANESE',
                   'LEXUS':'JAPANESE','MERCEDES':'EUROPEAN','MITSUBISHI':'JAPANESE',
                   'NISSAN':'JAPANESE','SKODA':'EUROPEAN','TOYOTA':'JAPANESE',
                   'VOLKSWAGEN':'EUROPEAN','VOLVO':'EUROPEAN'}
    PTS_dict = {'ORIGINAL': 'Оригинал', 'DUPLICATE': 'Дубликат'}
    wheel_dict = {'LEFT':'Левый', 'RIGHT':'Правый'}
    # bodyType
    df.dropna(subset=['bodyType'],inplace=True)
    df.bodyType = df.bodyType.apply(bodyType_unification)
    # color
    df.color = df.color.map(color_codes)
    # engineDisplacement
    df.engineDisplacement = df.engineDisplacement.apply(engineDisplacement_unification)
    # enginePower
    df.enginePower = df.enginePower.astype(int)
    # equipment_dict
    df['equipment_dict'] = df.Комплектация.apply(train_options)
    df.drop(columns=['Комплектация'],inplace=True)
    # modelDate 
    df.modelDate = df.modelDate.astype(int)
    # model_name
    df['model_name'] = df.model
    df.drop(columns=['model'],inplace=True)
    # numberOfDoors
    df.numberOfDoors = df.numberOfDoors.astype(int)
    # vehicleTransmission
    df.vehicleTransmission = df.vehicleTransmission.map(transmission_dict)
    # vendor
    df['vendor'] = df.brand.map(vendor_dict)
    # Владение
    df.Владение = df.Владение.apply(train_ownership_unification)
    # ПТС
    df.ПТС = df.ПТС.map(PTS_dict)
    # Руль
    df.Руль = df.Руль.map(wheel_dict)
    
    return df

## Установка параметров

In [6]:
marks_for_parsing = ['SKODA', 'AUDI', 'HONDA', 'VOLVO', 'BMW', 'NISSAN', 'INFINITI',
                     'MERCEDES', 'TOYOTA', 'LEXUS', 'VOLKSWAGEN', 'MITSUBISHI']
data_columns = ['bodyType', 'brand', 'car_url', 'color', 'engineDisplacement', 'enginePower', 
                'equipment_dict','fuelType', 'mileage', 'modelDate', 'model_name', 'numberOfDoors', 
                'productionDate', 'sell_id', 'vehicleTransmission', 'vendor', 
                'Владельцы', 'Владение', 'ПТС', 'Привод', 'Руль']
outer_datasets = ['all_auto_ru_09_09_2020.csv',
                     'all_car_27_08_2021_data_12_brands.csv',
                     'data1.csv',
                     'parsing_auto_ru.csv',
                     'train_df_full_part1.csv']
test_0909_uni_test = ['bodyType', 'brand', 'color', 'engineDisplacement', 'enginePower', 
                'equipment_dict','fuelType', 'mileage', 'modelDate', 'model_name', 'numberOfDoors', 
                'productionDate', 'vehicleTransmission', 'vendor', 
                'Владельцы', 'Владение', 'ПТС', 'Привод', 'Руль']
test_0909_uni_0909 = ['bodyType', 'brand', 'color', 'engineDisplacement', 'enginePower', 
                 'Комплектация','fuelType', 'mileage', 'modelDate', 'model', 'numberOfDoors', 
                'productionDate', 'vehicleTransmission', 
                'Владельцы', 'Владение', 'ПТС', 'Привод', 'Руль', 'price']

### Сокращение (перезапись только брендов из test)

In [41]:
# for df_name,pars_date in  zip(outer_datasets,parsing_date):
#     try:
#         df = pd.read_csv('./Preproject_7_data/' + df_name,low_memory=False)
#     except Exception:
#         df = pd.read_csv('./Preproject_7_data/' + df_name,sep='\t',low_memory=False) 
#     df = df[df.brand.apply(lambda x:x in marks_for_parsing)]
#     file_name = 'auto_ru_' + pars_date + '.csv'
#     print(f'./Project_7_data/{file_name}')
#     df.to_csv(f'./Project_7_data/{file_name}',index=False)

### Определение даты каждого датсета/ курса на эту дату

In [3]:
# parsing_date = ['2020_09_09','2021_08_29','2020_10_19','2021_08_13','2021_10_01']
# currency_rate = [1,1,1,1,1]
# basket_currency_rate_ = [1,1,1,1,1]

In [4]:
# datasets_to_train = []
# for df_name,pars_date in  zip(outer_datasets,parsing_date):
#     file_name = 'auto_ru_' + pars_date + '.csv'
#     datasets_to_train.append(file_name)

In [5]:
# datasets_to_train

### Корреспонденция колонок

### унификация с test  - auto_ru_2020_09_09.csv  с учетом data_columns

In [8]:
test = pd.read_csv('./Project_7_data/test.csv',low_memory=False)
df= pd.read_csv('./Preproject_7_data/all_auto_ru_09_09_2020.csv',low_memory=False)

In [9]:
test.columns

Index(['bodyType', 'brand', 'car_url', 'color', 'complectation_dict',
       'description', 'engineDisplacement', 'enginePower', 'equipment_dict',
       'fuelType', 'image', 'mileage', 'modelDate', 'model_info', 'model_name',
       'name', 'numberOfDoors', 'parsing_unixtime', 'priceCurrency',
       'productionDate', 'sell_id', 'super_gen', 'vehicleConfiguration',
       'vehicleTransmission', 'vendor', 'Владельцы', 'Владение', 'ПТС',
       'Привод', 'Руль', 'Состояние', 'Таможня'],
      dtype='object')

In [10]:
df.columns

Index(['bodyType', 'brand', 'color', 'fuelType', 'modelDate', 'name',
       'numberOfDoors', 'productionDate', 'vehicleConfiguration',
       'vehicleTransmission', 'engineDisplacement', 'enginePower',
       'description', 'mileage', 'Комплектация', 'Привод', 'Руль', 'Состояние',
       'Владельцы', 'ПТС', 'Таможня', 'Владение', 'price', 'start_date',
       'hidden', 'model'],
      dtype='object')

### Стандартизация датасетов

### Анализ распределения цены в 5(10) топ моделях

In [22]:
# for df_name in  datasets_to_train:
#     try:
#         df = pd.read_csv('./Preproject_7_data/' + df_name,low_memory=False)
#     except Exception:
#         df = pd.read_csv('./Preproject_7_data/' + df_name,sep='\t',low_memory=False) 
#     df = df[df.brand.apply(lambda x:x in marks_for_parsing)]
    
#     print(df.columns)
# #     models_for_distribution = df.model_name.value_counts().index[:5].to_list()
# #     print(models_for_distribution)

Index(['bodyType', 'brand', 'color', 'fuelType', 'modelDate', 'name',
       'numberOfDoors', 'productionDate', 'vehicleConfiguration',
       'vehicleTransmission', 'engineDisplacement', 'enginePower',
       'description', 'mileage', 'Комплектация', 'Привод', 'Руль', 'Состояние',
       'Владельцы', 'ПТС', 'Таможня', 'Владение', 'price', 'start_date',
       'hidden', 'model'],
      dtype='object')
Index(['Unnamed: 0', 'car_url', 'bodyType', 'brand', 'color',
       'complectation_dict', 'description', 'engineDisplacement',
       'enginePower', 'equipment_dict', 'fuelType', 'image', 'mileage',
       'modelDate', 'model_name', 'name', 'numberOfDoors', 'parsing_unixtime',
       'price', 'priceCurrency', 'productionDate', 'sell_id', 'views',
       'date_added', 'super_gen', 'vehicleConfiguration',
       'vehicleTransmission', 'Владельцы', 'Владение', 'ПТС', 'Привод', 'Руль',
       'Состояние', 'Таможня', 'region'],
      dtype='object')
Index(['bodyType', 'brand', 'car_url', 'col

In [20]:
models_for_distribution = df.model_name.value_counts().index[:5].to_list()
models_for_distribution

['Tiguan', 'X-Trail', 'Land Cruiser', 'Touareg', 'Polo']

### Соотношение курсового коэфициента с рапределением

In [10]:
'''for df_name in  datasets_to_train:
    
    try:
        df = pd.read_csv('./Preproject_7_data/' + df_name,low_memory=False)
#         print('ok')
    except:
        df = pd.read_csv('./Preproject_7_data/' + df_name,sep='\t',low_memory=False) 
#         print('trouble')
#     print(df.brand.unique())
    df = df[df.brand.apply(lambda x:x in marks_for_parsing)]
#     print(df.brand.unique())
    print(df_name,end=' ')
    try:
        print(date.fromtimestamp((df.parsing_unixtime.min())))
    except:
        print('no parsing_unixtime')
    print(df_name,end=' ')
    try:
        print(date.fromtimestamp((df.parsing_unixtime.max())))
    except:
        print('no parsing_unixtime')''';

## ==================================

In [27]:
def trfn(x):
    try:res = x.lower()   
    except Exception:res = x
    return res

df.bodyType.apply(trfn).unique()[:15]

array(['седан', 'универсал 5 дв.', 'хэтчбек 5 дв. sportback',
       'хэтчбек 3 дв.', 'хэтчбек 5 дв.', 'кабриолет', 'купе',
       'лифтбек sportback', 'лифтбек', 'седан long', 'внедорожник 5 дв.',
       'кабриолет roadster', 'седан 2 дв.', 'седан gran coupe',
       'компактвэн'], dtype=object)

In [ ]:
## ============================================

#### bodyType

%%time

def bodyType_unification(x):
    res=[]
    x = x.lower() if type(x) == str else x  # <================
    try:
        for body_type in test.bodyType.unique():
            if body_type in x:
                res.append(body_type)
    except Exception: return x
    return max(res)

df.dropna(subset=['bodyType'],inplace=True)
df.bodyType = df.bodyType.apply(bodyType_unification)

set(df.bodyType.unique()) == set(test.bodyType.unique())

#### brand

set(df.brand.unique()) == set(test.brand.unique())

#### color 

color_codes = {
    '040001': 'чёрный',
    'FAFBFB': 'белый', 
    '0000CC': 'синий', 
    '200204': 'коричневый', 
    'EE1D19': 'красный', 
    'CACECB': 'серый',
    'C49648': 'бежевый', 
    '97948F': 'серебристый', 
    'FFD600': 'жёлтый', 
    'FF8649': 'оранжевый', 
    '22A0F8': 'голубой',
    'FFC0CB': 'розовый', 
    'DEA522': 'золотистый', 
    '007F00': 'зелёный', 
    '660099': 'пурпурный',
    '4A2197': 'фиолетовый'}
df.color = df.color.map(color_codes)

set(test.color.unique()) == set(df.color.unique())

#### engineDisplacement

test.engineDisplacement = test.engineDisplacement.apply(
    lambda x: float(x[:-4]) if x[:-4] else np.NaN)   

def engineDisplacement_unification(x):
    x = float(re.sub("[^\d.]", r'', x)) if re.sub("[^\d.]", r'', x) else 0
    if x >= 7: x = 0
    return x

df.engineDisplacement = df.engineDisplacement.apply(engineDisplacement_unification)

#### enginePower

test.enginePower = test.enginePower.apply(lambda x: int(x[:-4]))

df.enginePower = df.enginePower.astype(int)

#### equipment_dict

test.equipment_dict = test.equipment_dict.apply(lambda x: list(json.loads(x).keys()) if x==x else [])

def train_options(x):
    point = "'available_options': "
    start = x.find(point)+len(point)+2
    finish = x.find("]",start) - 1
    return x[start:finish].split("', '")

df.Комплектация = df.Комплектация.apply(train_options)

#### fuelType

set(test.fuelType.unique()) == set(df.fuelType.unique())

#### mileage <===== ВСЕ ОК

#### modelDate

df.modelDate = df.modelDate.astype(int)

#### model_name

test_model = set(test.model_name.unique())

train_model = set(df.model.unique())

test_model - train_model

train_model - test_model

#### numberOfDoors

df.numberOfDoors = df.numberOfDoors.astype(int)

set(test.numberOfDoors.unique()) == set(df.numberOfDoors.unique())

#### productionDate

#### vehicleTransmission

transmission_dict = {
    'MECHANICAL':'механическая', 
    'AUTOMATIC':'автоматическая', 
    'ROBOT':'роботизированная', 
    'VARIATOR':'вариатор'}
df.vehicleTransmission = df.vehicleTransmission.map(transmission_dict)

set(test.vehicleTransmission.unique()) == set(df.vehicleTransmission.unique())

#### vendor 

vendor_dict = {
    'AUDI':'EUROPEAN',
    'BMW':'EUROPEAN',
    'HONDA':'JAPANESE',
    'INFINITI':'JAPANESE',
    'LEXUS':'JAPANESE',
    'MERCEDES':'EUROPEAN',
    'MITSUBISHI':'JAPANESE',
    'NISSAN':'JAPANESE',
    'SKODA':'EUROPEAN',
    'TOYOTA':'JAPANESE',
    'VOLKSWAGEN':'EUROPEAN',
    'VOLVO':'EUROPEAN'
}

df['vendor'] = df.brand.map(vendor_dict)

set(test.vendor.unique()) == set(df.vendor.unique())

#### Владельцы

owner_dict = {
    '3 или более': 3., 
    '2\xa0владельца': 2.,
    '1\xa0владелец': 1.
}
test.Владельцы = test.Владельцы.map(owner_dict)

set(test.Владельцы.unique()) , set(df.Владельцы.unique())

#### Владение

def test_ownership_unification(x):
    try:
        digits = re.findall('\d+',x) 
        if len(digits) == 2: res = int(digits[0])*12 + int(digits[1])
        elif len(digits) == 1 and 'месяц' in x: res = int(digits[0])
        elif len(digits) == 1 and 'месяц' not in x: res = int(digits[0])*12
    except Exception: res = 0   
    return res

test.Владение.apply(test_ownership_unification)

def train_ownership_unification(x):
    tmp = json.loads(x.replace("'",'"'))  if x==x else {'year': 2020, 'month': 9}
    res = (2020 - tmp['year'])*12 + tmp['month'] - 9
    if res<0: res = 0
    return res

df.Владение.apply(train_ownership_unification)

#### ПТС

test.ПТС.unique()

PTS_dict = {
    'ORIGINAL': 'Оригинал', 
    'DUPLICATE': 'Дубликат'
}
df.ПТС = df.ПТС.map(PTS_dict)

set(test.ПТС.unique()) ==  set(df.ПТС.unique())

#### Привод

set(test.Привод.unique()) == set(df.Привод.unique())

#### Руль

wheel_dict = {
    'LEFT':'Левый', 
    'RIGHT':'Правый'}

df.Руль = df.Руль.map(wheel_dict)

set(test.Руль.unique()) == set(df.Руль.unique())